Carga de librerías

In [1]:
# System management packages
import os
import re
import sys
sys.path.append('../src')

# Data science packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Geospatial packages
import h3
import contextily
import geopandas as gpd
from shapely import Polygon

# Personal packages
from settings import Settings
from ipywidgets import widgets

# Notebook settings
settings = Settings()

# Selección de fuente de datos

In [2]:
# Data directory and list of files
DATA_BASE_DIR = os.path.join(settings.ROOT, 'data')
RAW_DATA = os.path.join(DATA_BASE_DIR, 'datos-produccion-maiz')
#PROCESSED_DATA = os.path.join(DATA_BASE_DIR, 'maize_production_h3hex_cells')

LIST_OF_FILES = [file for file in os.listdir(RAW_DATA) if file.startswith('agg')]
H3_CATALOGUE = pd.read_csv(os.path.join(RAW_DATA, '01_h3_cells_catalogue.csv'))
H3_RESOLUTION_LIST = [col for col in H3_CATALOGUE.columns if col.startswith('hex')]

# Declare file selector instance and mesh resolution
file_selector = widgets.Dropdown(
    options=LIST_OF_FILES
    ,description='Files'
    ,disabled=False)

# Display selectors
display(file_selector)

Dropdown(description='Files', options=('agg-maize-panel-rcp2p6.csv', 'agg-maize-panel-rcp8p5.csv'), value='agg…

In [3]:
# Store file_selector output in variable
file_path = os.path.join(RAW_DATA, file_selector.value)

# Extract AIRCCA base model and h3 mesh resolution
get_aircca_model = re.compile(r'rcp[0-9]p[0-9]')
model_pointer = get_aircca_model.search(file_path).group()

print(f'Model pointer: {model_pointer.upper()}')

Model pointer: RCP2P6


In [4]:
# Load data into dataframe object
data = (
    pd.read_csv(file_path)
    .join(
        other=H3_CATALOGUE[['id'] + H3_RESOLUTION_LIST]
        ,on='id'
        ,how='left'
        ,rsuffix='__ignore'))

# Drop __ignore and format column names
data.drop(
    columns=[col for col in data.columns if col.__contains__('__ignore')]
    ,inplace=True)

data.columns = [
    re.sub(
        pattern=r'[-\. ]'
        ,repl='_'
        ,string=colname.lower().strip())
    for colname in data.columns]

# List of ordinary predictors, non geographical nor temporal data
ordinary_predictors = [var for var in data.columns if var.startswith('mean')]

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94224 entries, 0 to 94223
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             94224 non-null  int64  
 1   lon            94224 non-null  float64
 2   lat            94224 non-null  float64
 3   mean_precip    94224 non-null  float64
 4   mean_precip_2  94224 non-null  float64
 5   mean_temp      94224 non-null  float64
 6   mean_temp_2    94224 non-null  float64
 7   mean_yield     94224 non-null  float64
 8   period         94224 non-null  object 
 9   hex_0          94221 non-null  object 
 10  hex_1          94221 non-null  object 
 11  hex_2          94221 non-null  object 
 12  hex_3          94221 non-null  object 
 13  hex_4          94221 non-null  object 
 14  hex_5          94221 non-null  object 
 15  hex_6          94221 non-null  object 
 16  hex_7          94221 non-null  object 
 17  hex_8          94221 non-null  object 
 18  hex_9 

# Visualización inicial

In [5]:
def plot_yield_full_data(variable, period):
    # Visualization data
    _vis = (
        data
        .query(f"period == '{period}'"))

    # Figure config
    fig, ax = plt.subplots(figsize=(16,7))
    
    # Visualization elements
    map = ax.scatter(
        x=_vis.lon
        ,y=_vis.lat
        ,s=5
        ,c=_vis[variable]
        ,cmap='viridis')

    colorbar = plt.colorbar(map)
    
    # Annotations and styling
    ax.set_title(
        label=f'{model_pointer.upper()}, {period}: {variable}'
        ,fontsize=16)
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_frame_on(False)

    plt.show()

In [6]:
widgets.interact(
    plot_yield_full_data
    ,variable=widgets.Dropdown(
        options=ordinary_predictors
        ,value=ordinary_predictors[-1]
        ,description='Variable'
        ,disabled=False)
    ,period=widgets.Dropdown(
        options=data.period.unique()
        #,value=ordinary_predictors[-1]
        ,description='Period'
        ,disabled=False))

interactive(children=(Dropdown(description='Variable', index=4, options=('mean_precip', 'mean_precip_2', 'mean…

<function __main__.plot_yield_full_data(variable, period)>

# Visualización con h3-py

Revisar la siguiente [liga](https://github.com/uber/h3-py-notebooks/blob/master/notebooks/unified_data_layers.ipynb) para mayor detalle de las funcionalidades del framework de Uber.

Se empieza definiendo resoluciones de mallado para las celdas hexagonales

**IMPORTANTE**:

En la función `h3.latlng_to_cell(*, res)` el parámetro `res` pareciera que toma `res=15` como valor máximo, y no toma valores negativos. Pero el valor máximo pudiera deberse a recursos computacionales de mi máquina.

Adicionalmente, se define una función para extraer los polígonos asociados a cada uno de los hexágonos de la malla h3

In [7]:
def get_h3_polygon(row:str):
    points = h3.cell_to_boundary(
        h=row
        ,geo_json=True)

    return Polygon(points)

In [8]:
# Función de visualización interactiva
def plot_yield_h3(variable, resolution, period, basemap=False):
    _vis = (
        data
        .query(f"period == '{period}'")
        .filter(items=[resolution] + ordinary_predictors)
        # Group by hexagonal cell identifier
        .groupby(by=resolution)
        # Compute group sizes
        .mean()
        # Reset index for data processing
        .reset_index()
        # Compute groups centroids
        .assign(
            # Get centroid latitude
            lat = lambda _df: _df[resolution].apply(lambda row: h3.cell_to_latlng(row)[0])
            # Get centroid longitude
            ,lon = lambda _df: _df[resolution].apply(lambda row: h3.cell_to_latlng(row)[1])
            # Get h3 mesh polygons
            ,geometry = lambda _df: _df[resolution].apply(func=get_h3_polygon))
    )

    # Convert to geoDataFrame
    _vis = gpd.GeoDataFrame(data=_vis, crs='EPSG:4326')

    # Figure config
    fig, ax = plt.subplots(figsize=(16,7))
    
    # Visualization elements
    map = _vis.plot(
        column=variable
        ,legend=True
        ,ax=ax)

    # ---- Basemap
    if basemap:
        contextily.add_basemap(
            ax=ax
            ,crs=_vis.crs
            ,source=contextily.providers.CartoDB.PositronNoLabels)

    # Annotations and styling
    ax.set_title(
        label=f'{model_pointer.upper()}, {period}: {variable}'
        ,fontsize=16)
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_frame_on(False)

    plt.show()

In [9]:
widgets.interact(
    plot_yield_h3
    ,variable=widgets.Dropdown(
        options=ordinary_predictors
        ,value=ordinary_predictors[-1]
        ,description='Variable'
        ,disabled=False)
    ,resolution=widgets.Dropdown(
        options=H3_RESOLUTION_LIST
        ,value='hex_3'
        ,description='Resolution'
        ,disabled=False)
    ,period=widgets.Dropdown(
        options=data.period.unique()
        ,description='Period'
        ,disabled=False)
    ,basemap=widgets.Checkbox(
        value=True
        ,description='Basemap'
        ,disabled=False
        ,indent=True)
)

interactive(children=(Dropdown(description='Variable', index=4, options=('mean_precip', 'mean_precip_2', 'mean…

<function __main__.plot_yield_h3(variable, resolution, period, basemap=False)>

# Estrategias de modelado

El autor propone el siguiente modelo para la resolución del problema:

$$Y_{it} = C + \beta_1T_{it} + \beta_2T^2_{it} + \beta_3P_{it} + \beta_4P^2_{it} + \beta_5P_{it}T_{it} + \lambda_t + \alpha_i + u_{it}$$

En donde:

* $C$: Intercepto
* $T_{it}$: Temperatura del elemento $i$ al tiempo $t$
* $P_{it}$: Presipitación del elemento $i$ al tiempo $t$
* $\lambda_t$: Efectos fijos del periodo
* $\alpha_i$: Intercepto del elemento $i$
* $u_{it}$: Error del elemento $i$ al tiempo $t$